# Cross-match OGLE III and ZTF DR22

### using LINCC Frameworks' [LSDB](https://lsdb.io) and [SNAD](https://snad.space)'s OGLE III mirror

In [1]:
# Install LSDB
%pip install -U lsdb


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from functools import cache

import lsdb
import pandas as pd
from astropy.coordinates import Angle
from dask.distributed import Client

In [3]:
# Get OGLE III through SNAD's mirror
# I cache it to not hurt the server too much when re-running the code bellow

@cache
def get_ogle3_df():
    df = pd.read_csv(
        "https://ogle3.snad.space/api/v1/all",
        sep='\t',
        dtype_backend='pyarrow',
    )
    # Convert to decimal degrees
    df['ra'] = Angle(df['RA'], unit='hour').deg
    df['dec'] = Angle(df['Decl'], unit='deg').deg
    return df

In [ ]:
# Convert dataframe to LSDB Catalog
ogle3 = lsdb.from_dataframe(get_ogle3_df(), catalog_name="ogle3")
ogle3

In [ ]:
# Create ZTF DR22 Catalog object. No catalog data is read on this stage,
# metadata only is fetched

ztf_dr22 = lsdb.read_hats(
    # https://data.lsdb.io/#ZTF/ZTF_DR22
    'https://data.lsdb.io/hats/ztf_dr22/ztf_lc',
    margin_cache='https://data.lsdb.io/hats/ztf_dr22/ztf_lc_10arcs',
).nest_lists(
    # Make "nested" light-curve column from list-columns,
    # see nested-pandas pckage for more details:
    # https://nested-pandas.readthedocs.io/en/latest/
    list_columns=["hmjd", "mag", "magerr", "clrcoeff", "catflags"],  # light-curve columns
    base_columns=None,  # everything else is object metadta columns
    name="lc",
).query(
    # Select "good" observations
    "lc.catflags == 0",
)
ztf_dr22

In [ ]:
# Build cross-matched catalog
# Still no data fetched on this stage - just planning the future compute

xmatched = ogle3.crossmatch(
    ztf_dr22,
    radius_arcsec=1,
    # ZTF DR has multiple objects for the same sky source,
    # so we query for multiple matches
    n_neighbors=20,
    suffixes=["_ogle", "_ztf"],  # column suffixes
)
xmatched

In [ ]:
# Create Dask client to run things in parallel and run the compute
# It may took few hours and will produce a new HATS catalog.
# You can read it with LSDB or any other parquet tool such as Pandas,
# Dask, astropy, polars, etc.

with Client(n_workers=4, memory_limit="32GB", threads_per_worker=1) as client:
    # Use the link it gives to track the progress
    display(client)
    xmatched.to_hats("ogle3-x-ztf_dr22", catalog_name="ogle3-x-ztf_dr22")